# Naive Bayes text Document Classifier

<h3>Importing Libraries</h3>

In [1]:
import pandas as pd
import numpy as np
import nltk
import os

ModuleNotFoundError: No module named 'nltk'

In [2]:
#this block generates class_id, document_id, count of documents per class and a list containing paths to all the 
#files in the training dataset
directory = "train"
def pre_process(directory):
    paths = []              #contains paths to all the files of the dataset
    count_perclass = []  #contains count of documents in every class
    class_doc = []
    Class_id = {}           #assigns an id to each class
    doc_id_dict = {}        #assigns an id to each document
    id = 1
    doc_id = 1
    for subdir in os.listdir(directory):
        for document in os.listdir(directory+"/"+subdir):
            doc_id_dict[document] = doc_id
            doc_id+= 1
            class_doc.append([id , doc_id])
        Class_id[subdir] = id
        id+=1
        files = os.listdir(directory+'/'+subdir)
        paths.extend([ directory+'/'+subdir+'/'+i for i in files])
        count_perclass.append(len(files))
        
    class_doc = pd.DataFrame(class_doc) 
    
    return Class_id , doc_id_dict , class_doc , count_perclass , paths

In [5]:

from nltk.stem import LancasterStemmer

ModuleNotFoundError: No module named 'nltk'

In [5]:
#this block generates the total words dictionary and assigns a word ID to each word.
def generate_total_words():
    total_words = {}        #dictionary containing all the words as keys and their word id as values.
    for f in paths:
        file_words = []
        file = open(f , "r")
        for line in file.readlines():
            for word in line.split(" "):
                word = word.replace("\n" , "")
                word = word.replace("." , "")
                word = word.replace("," , "")
                word = word.replace(". " , "")
                word = word.replace(". " , "")
                word = word.replace(".\n" , "")
                word = word.replace(",\n" , "")
                word = word.replace("'s" , "")
                word = word.replace("s" , "")
                word = LancasterStemmer().stem(word)
                total_words[word] = 0

    w_id = 1
    for word in total_words.keys():
        total_words[word] = w_id
        w_id += 1
        
    return total_words

In [6]:
#this block generates a data list which contains information about every word in the dataset
def generate_data_df(directory , Class_id , doc_id_dict , total_words):
    data = []     #a list containing lists of all the words i.e [Class_id , document_id , total_words , word_frequency]
    for subdir in os.listdir(directory):
        pseudo_data = []
        #iterating over every document in every class
        for document in os.listdir(directory+"/"+subdir):
            word_frequency = {}

            file = open(directory + "/" + subdir + "/" + document , "r")

            #iterating over every line of every document in a class
            for line in file.readlines():
                #iterating over eery word of the line in a document
                for word in line.split(" "):
                    word = word.replace("\n" , "")
                    word = word.replace("." , "")
                    word = word.replace("," , "")
                    word = word.replace(". " , "")
                    word = word.replace(". " , "")
                    word = word.replace(".\n" , "")
                    word = word.replace(",\n" , "")
                    word = word.replace("'s" , "")
                    word = word.replace("s" , "")
                    word = LancasterStemmer().stem(word)

                    if word in word_frequency:
                        word_frequency[word] += 1
                    else:
                        word_frequency[word] = 0

            for word, count in word_frequency.items():
                pseudo_data = [Class_id[subdir] , doc_id_dict[document] , total_words[word] , count]
                data.append(pseudo_data)
                
    data_df = pd.DataFrame(data)
    data_df.rename(columns = {0:"Class_id" , 1:"document_id" , 2:"word_id" , 3:"word_frequency"} , inplace = True)
    
    return data_df

<h3>Pre-processing the training dataset to test against our naive bayes agent</h3>

In [4]:
Class_id , doc_id_dict , class_doc , count_perclass , paths = pre_process(directory)
total_words = generate_total_words()
data_df = generate_data_df(directory , Class_id , doc_id_dict , total_words)

NameError: name 'pre_process' is not defined

<h4>pre-prosssed dataframe</h4>

In [ ]:
data_df

,Class_id,document_id,word_id,word_frequency
0,1,1,1,8
1,1,1,2,3
2,1,1,3,0
3,1,1,4,0
4,1,1,5,0
...,...,...,...,...
198712,10,1000,1590,0
198713,10,1000,623,0
198714,10,1000,315,0
198715,10,1000,6192,0


In [ ]:
data_df.to_csv("data.csv")

<h3>Generating probability for every class</h3>

In [ ]:
#this block generates the probability of every class in the taining dataset
def generate_class_prob(directory):
    prob_class = {}         #contains probability of each class in the dataset
    for Class in os.listdir(directory):
        files = os.listdir(directory+'/'+Class)
        prob = len(files) / len(paths)
        prob_class[Class_id[Class]] = prob
    return prob_class

prob_class = generate_class_prob(directory)
prob_class

{1: 0.1,
 2: 0.1,
 3: 0.1,
 4: 0.1,
 5: 0.1,
 6: 0.1,
 7: 0.1,
 8: 0.1,
 9: 0.1,
 10: 0.1}

<h3>Applying laplace smoothing</h3>

In [ ]:
#with laplace smoothing
a = 1

#probability of each word given class
pb_ij = data_df.groupby(['Class_id','word_id'])
pb_j = data_df.groupby(['Class_id'])
Pr_a =  (pb_ij['word_frequency'].sum() + a) / (pb_j['word_frequency'].sum() + len(total_words))

Pr_a = Pr_a.unstack()

for c in range(1,11):
     Pr_a.loc[c,:] = Pr_a.loc[c,:].fillna(a/(pb_j['word_frequency'].sum()[c] + len(total_words)))

Pr_dict_a = Pr_a.to_dict()


<h4>probability of each word in a class</h4>

In [ ]:
Pr_a

word_id,1,2,3,4,5,6,7,8,9,10,...,31317,31318,31319,31320,31321,31322,31323,31324,31325,31326
Class_id,,,,,,,,,,,,,,,,,,,,,
1,0.002063,0.000861,0.000018,0.000018,0.000018,0.007443,0.000484,0.000484,0.000143,0.021487,...,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018
2,0.000067,0.000022,0.000022,0.000022,0.000022,0.007406,0.000313,0.000022,0.000022,0.014499,...,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022
3,0.000021,0.000727,0.000021,0.000021,0.000021,0.007030,0.000556,0.000021,0.000192,0.016582,...,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021
4,0.000475,0.000022,0.000022,0.000022,0.000022,0.007537,0.000194,0.000022,0.000259,0.015290,...,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022
5,0.000018,0.000037,0.000018,0.000037,0.000018,0.008046,0.000867,0.000018,0.000166,0.018030,...,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018
6,0.000012,0.000012,0.000012,0.000012,0.000012,0.008623,0.000012,0.000012,0.000122,0.023708,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
7,0.000012,0.000012,0.000012,0.000012,0.000012,0.342062,0.000012,0.000012,0.000012,0.009935,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
8,0.000020,0.000040,0.000020,0.000020,0.000020,0.175242,0.000020,0.000020,0.000020,0.008759,...,0.000020,0.000020,0.000020,0.000020,0.000020,0.000020,0.000020,0.000020,0.000020,0.000020
9,0.000014,0.000029,0.000014,0.000014,0.000014,0.211775,0.000014,0.000014,0.000130,0.011223,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014


<h4>Pr_a converted into dictionary</h4>

In [ ]:
Pr_dict_a

{1: {1: 0.002062595282934266,
  2: 6.712610758077508e-05,
  3: 2.1367977948246757e-05,
  4: 0.00047511067919231187,
  5: 1.845495146347765e-05,
  6: 1.2214188000781708e-05,
  7: 1.178508715071948e-05,
  8: 1.999800019998e-05,
  9: 1.4426059233399212e-05,
  10: 1.9362208840784556e-05},
 2: {1: 0.0008609093354856067,
  2: 2.2375369193591695e-05,
  3: 0.0007265112502403897,
  4: 2.1595939963286902e-05,
  5: 3.69099029269553e-05,
  6: 1.2214188000781708e-05,
  7: 1.178508715071948e-05,
  8: 3.999600039996e-05,
  9: 2.8852118466798424e-05,
  10: 1.9362208840784556e-05},
 3: {1: 1.7935611155950138e-05,
  2: 2.2375369193591695e-05,
  3: 2.1367977948246757e-05,
  4: 2.1595939963286902e-05,
  5: 1.845495146347765e-05,
  6: 1.2214188000781708e-05,
  7: 1.178508715071948e-05,
  8: 1.999800019998e-05,
  9: 1.4426059233399212e-05,
  10: 1.9362208840784556e-05},
 4: {1: 1.7935611155950138e-05,
  2: 2.2375369193591695e-05,
  3: 2.1367977948246757e-05,
  4: 2.1595939963286902e-05,
  5: 3.6909902926955

<h3>Probability without laplace smoothing</h3>

In [ ]:
#without laplace smoothing

#probability of each word given class
pb_kl = data_df.groupby(['Class_id','word_id'])
pb_l = data_df.groupby(['Class_id'])
Pr =  (pb_kl['word_frequency'].sum()) / (pb_l['word_frequency'].sum())

Pr = Pr.unstack()

for c in range(1,11):
  Pr.loc[c,:] = Pr.loc[c,:]

Pr_dict = Pr.to_dict()

<h4>probability of each word in a class</h4>

In [ ]:
Pr

word_id,1,2,3,4,5,6,7,8,9,10,...,31317,31318,31319,31320,31321,31322,31323,31324,31325,31326
Class_id,,,,,,,,,,,,,,,,,,,,,
1,0.004667,0.001924,0.0,0.000000,0.0,0.016947,0.001064,0.001064,0.000287,0.048999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000150,NaN,NaN,NaN,NaN,0.024690,0.000973,0.000000,0.000000,0.048406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.002133,NaN,0.000000,NaN,0.021198,0.001616,NaN,0.000517,0.050087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.001402,0.000000,NaN,0.000000,NaN,0.023233,0.000534,0.000000,0.000734,0.047199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.000000,0.000044,NaN,0.000044,NaN,0.019029,0.002012,0.000000,0.000350,0.042695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000,0.000000,NaN,NaN,NaN,0.013948,NaN,NaN,0.000178,0.038381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000000,0.000000,NaN,NaN,NaN,0.542231,0.000000,0.000000,0.000000,0.015730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0.000054,NaN,NaN,NaN,0.469083,NaN,NaN,0.000000,0.023395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,0.000026,NaN,0.000000,NaN,0.386361,0.000000,NaN,0.000211,0.020451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Pr_a converted into dictionary</h4>

In [ ]:
Pr_dict

{1: {1: 0.004666584796757952,
  2: 0.00014963339817447254,
  3: 0.0,
  4: 0.0014019627478469858,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: nan,
  9: nan,
  10: 0.0},
 2: {1: 0.0019239428548037168,
  2: nan,
  3: 0.002132747366380146,
  4: 0.0,
  5: 4.374453193350831e-05,
  6: 0.0,
  7: 0.0,
  8: 5.35360565340757e-05,
  9: 2.6320638012265417e-05,
  10: 0.0},
 3: {1: 0.0,
  2: nan,
  3: nan,
  4: nan,
  5: nan,
  6: nan,
  7: nan,
  8: nan,
  9: nan,
  10: nan},
 4: {1: 0.0,
  2: nan,
  3: 0.0,
  4: 0.0,
  5: 4.374453193350831e-05,
  6: nan,
  7: nan,
  8: nan,
  9: 0.0,
  10: nan},
 5: {1: 0.0,
  2: nan,
  3: nan,
  4: nan,
  5: nan,
  6: nan,
  7: nan,
  8: nan,
  9: nan,
  10: nan},
 6: {1: 0.01694707110401572,
  2: 0.02468951069878797,
  3: 0.02119821624765721,
  4: 0.02323252553575005,
  5: 0.019028871391076115,
  6: 0.01394769121196534,
  7: 0.5422310235955686,
  8: 0.4690829273515713,
  9: 0.38636064538204407,
  10: 0.2204123812804488},
 7: {1: 0.0010643088132956731,
  2: 0.0009726170881

In [ ]:
#Common stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/shivansh/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


<h3>Removing Stopwords</h3>

In [ ]:
total_words_df = pd.DataFrame(list(total_words.items()))

#word_id of all words
total = set(total_words_df[1])

#generating set of good words
total_words_df = total_words_df[~total_words_df[0].isin(stop_words)]
good = list(total_words_df[1])
good = set(good)

#generating set of stop words
stop = total - good

for bad in stop:
    for j in range(1,11):
        Pr_dict[j][bad] = a/(pb_j['word_frequency'].sum()[j] + len(total_words))    #removing stop words
        

<h4>dataframe of all the words after removing stop words</h4>

In [ ]:
total_words

<h3>Calculating probability of every class given document of test dataset and then assigning class to every document</h3>

In [ ]:
def assign_class(Pr_dict , dict):
    result = []
    for doc_id in range(1, len(dict)+1):
        prob_dict = {}
        for class_id in range(1,len(count_perclass) + 1):
            prob_dict[class_id] = 1
            for word_id in dict[doc_id]: 
                prob=Pr_dict[word_id][class_id]
                if prob != 0:            
                    prob_dict[class_id]+=(np.log(1+ dict[doc_id][word_id]))*np.log(prob)
                else:
                    prob_dict[class_id] += 0       
#calculating final probability of each word      
            prob_dict[class_id] +=  np.log(prob_class[class_id])                          
            
#finding maximum value of probability
        max_probability = max(prob_dict, key=prob_dict.get)
        result.append(max_probability)

    return result

<h2>Creating a new dictionary to store the frequency, doc_id and word_count and to provide a way to see the results with and without laplace smoothing </h2>

In [ ]:
def algorithm(df , laplace_smoothing= False):

    df_dict = df.to_dict()
#new_dict is a dictionary containing keys as document id's and values as dictionaries containing wordId as keys
#and word frequency as values.
    new_dict = {}
    
    for ID in range(len(df_dict['document_id'])):
        doc_id = df_dict['document_id'][ID]
        word_id = df_dict['word_id'][ID]
        frequency = df_dict['word_frequency'][ID]
        try: 
            new_dict[doc_id][word_id] = df_dict['word_frequency'][ID] 
        except:
            new_dict[df_dict['document_id'][ID]] = {}
            new_dict[doc_id][word_id] = df_dict['word_frequency'][ID]
  
    return assign_class(Pr_dict_a , new_dict) if laplace_smoothing else assign_class(Pr_dict , new_dict)

<h3>Testing our model against the training dataset and calculating the error</h3>

In [ ]:
test_data_df = data_df

classification_withLaplace = algorithm(test_data_df , True)
classification_withoutLaplace = algorithm(test_data_df , False)

#training dataframe for verification of our model

training_data = list(class_doc[0]) 
correctness = 0
correctness_a = 0

for x,y in zip(classification_withoutLaplace, training_data):
    if x != y:
        correctness +=1
    else:
        pass   
print("Error without laplace:\t\t", "{0:.5f}".format(correctness/ len(training_data) *100), "%")

for x,y in zip(classification_withLaplace, training_data):
    if x != y:
        correctness_a +=1
    else:
        pass 
print("Error with laplace:\t\t", "{0:.5f}".format(correctness_a/ len(training_data) *100) , "%")